In [5]:
import pandas as pd

In [6]:
df = pd.read_csv('data/small_corpus.csv')

In [18]:
val = df.head(1)['reviews'].item()

In [19]:
val

'Recently UBISOFT had to settle a huge class-action suit brought against the company for bundling (the notoriously harmful) StarFORCE DRM with its released games. So what the geniuses at the helm do next? They decide to make the same mistake yet again - by choosing the same DRM scheme that made BIOSHOCK, MASS EFFECT and SPORE infamous: SecuROM 7.xx with LIMITED ACTIVATIONS!\n\nMASS EFFECT can be found in clearance bins only months after its release; SPORE not only undersold miserably but also made history as the boiling point of gamers lashing back, fed up with idiotic DRM schemes. And the clueless MBAs that run an art-form as any other commodity business decided that, "hey, why not jump into THAT mud-pond ourselves?"\n\nThe original FAR CRY was such a GREAT game that any sequel of it would have to fight an uphill battle to begin with (especially without its original developing team). Now imagine shooting this sequel on the foot with a well known, much hated and totally useless DRM sch

In [23]:
from transformers import pipeline
classifier = pipeline('sentiment-analysis')

In [24]:
output = classifier(val[:512])

In [29]:
reviews = list(df["reviews"])

In [31]:
len(reviews[0])

2925

In [32]:
output=classifier([reviews[0][:512],reviews[1]])

In [33]:
df

[{'label': 'NEGATIVE', 'score': 0.9991271495819092},
 {'label': 'NEGATIVE', 'score': 0.9996480345726013}]